In [1]:
from UTILITY_quickstart import *

In [2]:
#Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Not using setLattice(). Golden lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


## Sensitivity to moves

In [3]:
# results = {"quadOffset": [], "beamOffset": []}
# step = 50e-6
# for quadOffset in np.arange(-200e-6, 200e-6 + step, step):
#     setXOffset(tao, "Q5FF", quadOffset)
#     beamOffset = getMatrix(tao,"MFFF","PENT",order=0)[0]

    
#     results["quadOffset"].append(quadOffset)
#     results["beamOffset"].append(beamOffset)
#     #print([L2PhaseSet, P["mean_energy"]])

# plt.scatter(
#     1e6 * np.array(results["quadOffset"]), 
#     1e6 * np.array(results["beamOffset"])
# )

# # Adding title and labels
# plt.xlabel('Q5FF offset [um]')
# plt.ylabel('Beam offset at PENT [um]')

# # Display the plot
# plt.show()

In [4]:
# #Show all cases

# quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"]


# for quad in quadNameList: 
#     #Start from golden lattice
#     tao = initializeTao(
#         loadDefaultLatticeTF = False
#     )
    
    
#     results = {"quadOffset": [], "beamOffset": []}
#     step = 50e-6
#     for quadOffset in np.arange(0, 200e-6 + step, step):
#         setXOffset(tao, quad, quadOffset)
#         beamOffset = np.sqrt( getMatrix(tao,"MFFF","PENT",order=0)[0] ** 2 + getMatrix(tao,"MFFF","PENT",order=0)[2] ** 2 ) 
    
        
#         results["quadOffset"].append(quadOffset)
#         results["beamOffset"].append(beamOffset)
#         #print([L2PhaseSet, P["mean_energy"]])

    
#     # plt.scatter(
#     #     1e6 * np.array(results["quadOffset"]), 
#     #     1e6 * np.array(results["beamOffset"])
#     # )
    
#     # # Adding title and labels
#     # plt.xlabel(f'{quad} offset [um]')
#     # plt.ylabel('Beam offset at PENT [um]')

#     print(f"""{quad} : {results['beamOffset'][-1] / results['quadOffset'][-1]}""")
    
#     # Display the plot
#     plt.show()

# Automating

In [5]:
#Load a default configuration
importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

#Initialize tao object
tao = initializeTao(
    #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
    #numMacroParticles = 1e4,
    #csrTF = False
)

#Set lattice to the loaded config
setLattice(tao, **importedDefaultSettings)

tao.twiss_at_s(ele = "PENT", s_offset = 0.0)

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Overwriting lattice with setLattice() defaults
No defaults file provided to setLattice(). Using setLattice_configs/defaults.yml
WARNING! No beam file is specified!
Number of macro particles defined by input file


{'beta_a': 0.50649204966748,
 'alpha_a': 0.0466683356262582,
 'gamma_a': 1.9786646882376,
 'phi_a': 73.0782192378143,
 'eta_a': -5.53984755575948e-06,
 'etap_a': -1.05998552673305e-05,
 'beta_b': 0.500627846338041,
 'alpha_b': -0.120964597934503,
 'gamma_b': 2.02671993053367,
 'phi_b': 60.5521347355512,
 'eta_b': -2.11809896530596e-18,
 'etap_b': -2.52217362743306e-18,
 'c_mat11': 1.24354101578855e-16,
 'c_mat12': -2.15263057112982e-17,
 'c_mat21': -3.33737707176956e-17,
 'c_mat22': 4.78887427948486e-18,
 'gamma_c': 1.0}

In [6]:
allMagnetConfigs = []

betaSetting = 0.10

for offset in [-2*betaSetting, 0, 2*betaSetting]:
    allMagnetConfigs.append(
        finalFocusSolver(tao, 
                     ele = "PENT",
                     s_offset = offset,
                     targetBetaX = betaSetting,
                     targetAlphaX = 0.0,
                     targetBetaY = betaSetting,
                     targetAlphaY = 0,
                     verbose = True
                     )
    )

allMagnetConfigs = { "upstream" : allMagnetConfigs[0], "PENT" : allMagnetConfigs[1], "downstream" : allMagnetConfigs[2] }
waistSOffset = { "upstream" : -2*betaSetting, "PENT" : 0, "downstream" : 2*betaSetting }

Optimization Results:
Optimal Parameters: [-107.09040479  -77.86315435   59.20417671  149.47806587 -243.14004919
  134.70989378]
Objective Function Value at Optimal Parameters: 0.003834730557100828
Number of Iterations: 777
Converged: False
Optimization Results:
Optimal Parameters: [-104.75621701  -86.0994063    62.33199778  147.40186803 -239.71209879
  131.74425868]
Objective Function Value at Optimal Parameters: 0.003322416294427039
Number of Iterations: 771
Converged: False
Optimization Results:
Optimal Parameters: [-110.08130882  -87.95379178   71.38540807  140.13195531 -237.46380002
  140.25745045]
Objective Function Value at Optimal Parameters: 0.0030183496348305913
Number of Iterations: 788
Converged: False


In [7]:
setLattice(tao, ** ( importedDefaultSettings | allMagnetConfigs["downstream"] ))
quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"]
# for quad in quadNameList:
#     #print(tao.ele_gen_attribs(quad)["K1"])
#     print(getMatrix(tao, quad, "PENT")[0][1])

tao.twiss_at_s(ele = "PENT", s_offset = waistSOffset["downstream"])

{'beta_a': 0.114894548871531,
 'alpha_a': 9.09302570062387e-05,
 'gamma_a': 8.70363318442949,
 'phi_a': 73.0743384172971,
 'eta_a': -2.62810388596073e-06,
 'etap_a': -2.3414051165625e-05,
 'beta_b': 0.111205786493467,
 'alpha_b': 8.38086142518346e-05,
 'gamma_b': 8.9923378859663,
 'phi_b': 60.830951104676,
 'eta_b': 1.52337686912451e-17,
 'etap_b': 2.35309963926359e-18,
 'c_mat11': 1.26571517551516e-16,
 'c_mat12': 1.51238510677849e-17,
 'c_mat21': -9.15654841883837e-17,
 'c_mat22': -1.11879404443835e-17,
 'gamma_c': 1.0}

## Monte Carlo

In [8]:
# quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"]
# #quadErrorSigma = 10e-6 


# results = pd.DataFrame()


# for quadErrorSigma in [1e-6, 2e-6, 5e-6, 10e-6, 20e-6, 50e-6, 100e-6, 200e-6, 500e-6]:
# #for quadErrorSigma in [50e-6]:
#     for iteration in np.arange(20):

#         #Prevent recalculation until changes are made
#         tao.cmd("set global lattice_calc_on = F")
        
#         for quad in quadNameList:
#             setXOffset(tao, quad, quadErrorSigma * np.random.normal())
#             setYOffset(tao, quad, quadErrorSigma * np.random.normal())
            
#         #Reenable lattice calculations
#         tao.cmd("set global lattice_calc_on = T")

#         for magnetConfigKey in allMagnetConfigs.keys():
#             magnetConfig = allMagnetConfigs[magnetConfigKey]
            
#             setLattice(tao, **(importedDefaultSettings | magnetConfig ))
            
#             newLine = { "quadErrorSigma" : quadErrorSigma, "iteration": iteration, "magnetConfigKey" : magnetConfigKey } 

#             for evalEle in ["PENT"]: #["M5FF", "IPOTR1", "PENT", "IPOTR2", "M0EX"]:

#                 # newLine[f"{evalEle}_xCenter"] = getMatrix(tao, "MFFF", evalEle, order=0)[0]
#                 # newLine[f"{evalEle}_yCenter"] = getMatrix(tao, "MFFF", evalEle, order=0)[2]


#                 #Evaluate at element
#                 newLine[f"{evalEle}_xCenter"] = tao.orbit_at_s(ele = evalEle, s_offset = 0)["x"]
#                 newLine[f"{evalEle}_yCenter"] = tao.orbit_at_s(ele = evalEle, s_offset = 0)["y"]

#                 #Evaluate at waist position
#                 newLine[f"{evalEle}_offset_xCenter"] = tao.orbit_at_s(ele = evalEle, s_offset = waistSOffset["downstream"])["x"]
#                 newLine[f"{evalEle}_offset_yCenter"] = tao.orbit_at_s(ele = evalEle, s_offset = waistSOffset["downstream"])["y"]
        
        
#             # Convert newLine to a DataFrame before concatenating
#             newLine_df = pd.DataFrame([newLine])
#             results = pd.concat([results, newLine_df], ignore_index=True)

# # Save results to JSON
# results.to_json('results.json', orient='records')

## Single magnet sensitivities

In [9]:
quadNameList = ["Q5FF", "Q4FF", "Q3FF", "Q2FF", "Q1FF", "Q0FF"]
#quadErrorSigma = 10e-6 


results = pd.DataFrame()

#     #Start from golden lattice
tao = initializeTao(
    loadDefaultLatticeTF = False
)

#for quadErrorSigma in [1e-6, 2e-6, 5e-6, 10e-6, 20e-6, 50e-6, 100e-6, 200e-6, 500e-6]:
for quad in quadNameList:
    for axis in ["X", "Y"]: 
        
        #Prevent recalculation until changes are made
        tao.cmd("set global lattice_calc_on = F")
        
        # for quad in quadNameList:
        #     setXOffset(tao, quad, quadErrorSigma * np.random.normal())
        #     setYOffset(tao, quad, quadErrorSigma * np.random.normal())


        #Clean start each loop (remember that setLattice() doesn't check for these offsets!!)
        for quadInnerLoop in quadNameList:
            setXOffset(tao, quadInnerLoop, 0)
            setYOffset(tao, quadInnerLoop, 0)

        tao.cmd(f"set ele {quad} {axis}_OFFSET = {50e-6}")
        
        #Reenable lattice calculations
        tao.cmd("set global lattice_calc_on = T")



        
        for magnetConfigKey in allMagnetConfigs.keys():
            magnetConfig = allMagnetConfigs[magnetConfigKey]
            
            setLattice(tao, **(importedDefaultSettings | magnetConfig ))
            
            newLine = { "quad" : quad, "axis": axis, "magnetConfigKey" : magnetConfigKey } 

            #for evalEle in ["M5FF", "IPOTR1", "PENT", "IPOTR2", "M0EX"]:
            for evalEle in ["PENT"]:

                newLine[f"{evalEle}_xCenter"] = tao.ele_orbit("PENT")['x']
                newLine[f"{evalEle}_yCenter"] = tao.ele_orbit("PENT")['y']
        
        
            # Convert newLine to a DataFrame before concatenating
            newLine_df = pd.DataFrame([newLine])
            results = pd.concat([results, newLine_df], ignore_index=True)

# Save results to JSON
results.to_json('results.json', orient='records')

Environment set to:  /Users/nmajik/Documents/SLAC/FACET2-Bmad-PyTao
Tracking to end
CSR off
Not using setLattice(). Golden lattice
WARNING! No beam file is specified!
Number of macro particles defined by input file


## Printing R-matrices

In [10]:
# #     #Start from golden lattice
# tao = initializeTao(
#     loadDefaultLatticeTF = False
# )

# print("\n\n\n")
# for magnetConfigKey in allMagnetConfigs.keys():
#     magnetConfig = allMagnetConfigs[magnetConfigKey]

#     print(f"Waist location is {magnetConfigKey}")
#     setLattice(tao, **(importedDefaultSettings | magnetConfig ))

#     for quad in quadNameList:
#         print(quad)
#         rawMat6 = tao.ele_mat6(quad)
#         df = pd.DataFrame.from_dict(rawMat6, orient='index', columns=['1', '2', '3', '4', '5', '6'])
#         displayMatrix(df)

#     print("\n\n\n")

## Sensitivity to incoming beam errors

In [11]:
# #Load a default configuration
# importedDefaultSettings = loadConfig("setLattice_configs/defaults.yml")

# #Initialize tao object
# tao = initializeTao(
#     #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
#     #numMacroParticles = 1e4,
#     #csrTF = False
# )

# #Set lattice to the loaded config
# setLattice(tao, **importedDefaultSettings)

# tao.twiss_at_s(ele = "PENT", s_offset = 0.0)

In [12]:
# results = {"incomingError": [], "beamOffset": []}
# step = 50e-6
# for incomingError in np.arange(-200e-6, 200e-6 + step, step):
#     #setXOffset(tao, "Q5FF", quadOffset)

#     tao.cmd(f"set ele patch_sector20 x_offset = {incomingError}")
#     beamOffset = tao.ele_orbit("PENT")['x']
    
#     results["incomingError"].append(incomingError)
#     results["beamOffset"].append(beamOffset)
#     #print([L2PhaseSet, P["mean_energy"]])

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffset"])
# )

# # Adding title and labels
# plt.xlabel('Incoming error [um]')
# plt.ylabel('Beam offset at PENT [um]')

# # Display the plot
# plt.show()

In [13]:
# allMagnetConfigs = []

# for offset in [-0.5, 0, 0.5]:
#     allMagnetConfigs.append(
#         finalFocusSolver(tao, 
#                      ele = "PENT",
#                      s_offset = offset,
#                      targetBetaX = 0.5,
#                      targetAlphaX = 0.0,
#                      targetBetaY = 0.5,
#                      targetAlphaY = 0,
#                      verbose = True
#                      )
#     )

# allMagnetConfigs = { "upstream" : allMagnetConfigs[0], "PENT" : allMagnetConfigs[1], "downstream" : allMagnetConfigs[2] }

In [14]:
# #Initialize tao object
# tao = initializeTao(
#     #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
#     #numMacroParticles = 1e4,
#     #csrTF = False
# )

# #Set lattice to the loaded config
# setLattice(tao, **importedDefaultSettings)


# results = {"incomingError": [], "beamOffsetUpstream": [], "beamOffsetPENT": [], "beamOffsetDownstream": []}
# axis = "y"

# step = 50e-6
# for incomingError in np.arange(-200e-6, 200e-6 + step, step):
#     #setXOffset(tao, "Q5FF", quadOffset)

#     tao.cmd(f"set ele patch_sector20 {axis}_offset = {incomingError}")

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["upstream"] ))
#     beamOffsetUpstream = tao.ele_orbit("PENT")[axis]

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["PENT"] ))
#     beamOffsetPENT = tao.ele_orbit("PENT")[axis]
    
#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["downstream"] ))
#     beamOffsetDownstream = tao.ele_orbit("PENT")[axis]
    
#     results["incomingError"].append(incomingError)
#     results["beamOffsetUpstream"].append(beamOffsetUpstream)
#     results["beamOffsetPENT"].append(beamOffsetPENT)
#     results["beamOffsetDownstream"].append(beamOffsetDownstream)
#     #print([L2PhaseSet, P["mean_energy"]])

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetUpstream"]),
#     label = "waist = PENT - 50 cm"
# )

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetPENT"]),
#     label = "waist = PENT"
# )

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetDownstream"]),
#     label = "waist = PENT + 50 cm"
# )

# # Adding title and labels
# plt.xlabel(f'Incoming {axis} error [um]')
# plt.ylabel(f'Beam {axis} offset at PENT [um]')

# plt.legend(loc="upper left")

# # Display the plot
# plt.show()

In [15]:
# #Initialize tao object
# tao = initializeTao(
#     #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
#     #numMacroParticles = 1e4,
#     #csrTF = False
# )

# #Set lattice to the loaded config
# setLattice(tao, **importedDefaultSettings)


# results = {"incomingError": [], "beamOffsetUpstream": [], "beamOffsetPENT": [], "beamOffsetDownstream": []}
# axis = "x"

# step = 5e-6
# for incomingError in np.arange(-20e-6, 20e-6 + step, step):
#     #setXOffset(tao, "Q5FF", quadOffset)

#     tao.cmd(f"set ele patch_sector20 {axis}_pitch = {incomingError}")

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["upstream"] ))
#     beamOffsetUpstream = tao.ele_orbit("PENT")[axis]

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["PENT"] ))
#     beamOffsetPENT = tao.ele_orbit("PENT")[axis]
    
#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["downstream"] ))
#     beamOffsetDownstream = tao.ele_orbit("PENT")[axis]
    
#     results["incomingError"].append(incomingError)
#     results["beamOffsetUpstream"].append(beamOffsetUpstream)
#     results["beamOffsetPENT"].append(beamOffsetPENT)
#     results["beamOffsetDownstream"].append(beamOffsetDownstream)
#     #print([L2PhaseSet, P["mean_energy"]])

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetUpstream"]),
#     label = "waist = PENT - 50 cm"
# )

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetPENT"]),
#     label = "waist = PENT"
# )

# plt.scatter(
#     1e6 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetDownstream"]),
#     label = "waist = PENT + 50 cm"
# )

# # Adding title and labels
# plt.xlabel(f"Incoming {axis}' error [urad]")
# plt.ylabel(f'Beam {axis} offset at PENT [um]')

# plt.legend(loc="upper left")

# # Display the plot
# plt.show()

In [16]:
# #Initialize tao object
# tao = initializeTao(
#     #inputBeamFilePathSuffix = importedDefaultSettings["inputBeamFilePathSuffix"],
#     #numMacroParticles = 1e4,
#     #csrTF = False
# )

# #Set lattice to the loaded config
# setLattice(tao, **importedDefaultSettings)


# results = {"incomingError": [], "beamOffsetUpstream": [], "beamOffsetPENT": [], "beamOffsetDownstream": []}
# axis = "y"

# step = 10e6
# for incomingError in np.arange(9.9e9, 10.1e9 + step, step):
#     #setXOffset(tao, "Q5FF", quadOffset)

#     tao.cmd(f"set ele patch_sector20 e_tot_set = {incomingError}")

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["upstream"] ))
#     beamOffsetUpstream = tao.ele_orbit("PENT")[axis]

#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["PENT"] ))
#     beamOffsetPENT = tao.ele_orbit("PENT")[axis]
    
#     setLattice(tao, **(importedDefaultSettings | allMagnetConfigs["downstream"] ))
#     beamOffsetDownstream = tao.ele_orbit("PENT")[axis]
    
#     results["incomingError"].append(incomingError)
#     results["beamOffsetUpstream"].append(beamOffsetUpstream)
#     results["beamOffsetPENT"].append(beamOffsetPENT)
#     results["beamOffsetDownstream"].append(beamOffsetDownstream)
#     #print([L2PhaseSet, P["mean_energy"]])

# plt.scatter(
#     1e-9 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetUpstream"]),
#     label = "waist = PENT - 50 cm"
# )

# plt.scatter(
#     1e-9 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetPENT"]),
#     label = "waist = PENT"
# )

# plt.scatter(
#     1e-9 * np.array(results["incomingError"]), 
#     1e6 * np.array(results["beamOffsetDownstream"]),
#     label = "waist = PENT + 50 cm"
# )

# # Adding title and labels
# plt.xlabel(f"Incoming energy [GeV]")
# plt.ylabel(f'Beam {axis} offset at PENT [um]')

# plt.legend(loc="upper left")

# # Display the plot
# plt.show()